In [1]:
from tensorflow.keras.layers import Conv2D, UpSampling2D, BatchNormalization,Dense, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, Dropout, AveragePooling2D, MaxPooling2D
from tensorflow.keras.layers import Add, GlobalAveragePooling2D, Reshape, Dense, multiply

from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import shutil

import numpy as np 
import cv2
from glob import glob
from tqdm import tqdm
import time
import matplotlib.pyplot as plt

import json
import os

from sklearn.model_selection import train_test_split
from imshowtools import imshow
from albumentations import  CoarseDropout, RandomBrightness, RandomContrast, HorizontalFlip
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score, cohen_kappa_score, matthews_corrcoef

C:\Users\ROG\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\ROG\anaconda3\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're

In [2]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [3]:
def segmentation_task(model_path, dataset_path, save_path):
    """This function will do the segmentation task
    model_path: path to the model you want to load to process the segmentation.
    dataset_path: path to the images that you want to apply the segmentation on.
    save_path: where you want to save the output.
    
    
    """
    model = tf.keras.models.load_model(model_path)
    print('Model loaded successfully')
    
    test_x = sorted(glob(os.path.join(r'',dataset_path, '*')))
    print(f"Test Images: {len(test_x)}")
    
    create_dir(save_path)
    time_taken = []

    for x in tqdm(test_x):
        name = x.split('\\')[-1]

        x = cv2.imread(x, cv2.IMREAD_COLOR)
        x = cv2.resize(x, (256,256), interpolation=cv2.INTER_LINEAR)
        x = x / 255.0
        x = np.expand_dims(x, axis=0)

        start_time = time.time()
        p = model.predict(x)[0]
        total_time = time.time() - start_time
        time_taken.append(total_time)

        p = p > 0.6
        p = p * 255

        cv2.imwrite(os.path.join(save_path, name), p)
    print('segmented images saved successfully')

In [4]:
def model_perfromance_check(pred_mask_dir, true_mask_dir, cup_or_disc):
    """This function will check the model perfromance by applying various metrices to the model such
    as DICE, PIXEL ACCURACY, RECALL, JACCARD INDEX, COHEN KAPPA, PRECEISION, AND 
    MATTHEWS CORRELATION COEFFICIENT.
    
    pred_mask_dir : is the directory to the predected masks by the model, 
    true_mask_dir : is the directory to the true masks,
    cup_or_disc : if you use this function to check for cup model accuracy set it as cup otherwise set
    it as disc"""
    
    print(f"The performance for {cup_or_disc}'s model")



    # Load filenames
    pred_mask_files = sorted(glob(os.path.join(pred_mask_dir, '*')))
    true_mask_files = sorted(glob(os.path.join(true_mask_dir, '*')))
    print('predicted masks',len(pred_mask_files),'-', 'true masks' ,len(true_mask_files))
    # Convert list of true mask files into a dictionary for quick access
    true_mask_map = {os.path.basename(f).replace(f'.jpg_optic_{cup_or_disc}_mask.png', ''): f for f in true_mask_files}
    score = []

    # Process each predicted mask
    for pred_path in tqdm(pred_mask_files):
        base_name = os.path.basename(pred_path).rsplit('.', 1)[0]
        true_mask_filename = base_name + f'_optic_{cup_or_disc}_mask.png'
        true_path = true_mask_map.get(base_name)

        if not true_path:
            #print(f"No matching true mask found for predicted mask: {base_name}")
            continue

        # Read and process the predicted mask
        pred_mask = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
        pred_mask = (pred_mask / 255.0) > 0.5
        pred_mask = pred_mask.astype(np.int32).flatten()

        # Read and process the true mask
        true_mask = cv2.imread(true_path, cv2.IMREAD_GRAYSCALE)
        true_mask = (true_mask / 255.0) > 0.5
        true_mask = true_mask.astype(np.int32).flatten()

        # Calculate metrics
        acc_value = accuracy_score(true_mask, pred_mask)
        f1_value = f1_score(true_mask, pred_mask, labels=[0,1], average='binary')
        jac_value = jaccard_score(true_mask, pred_mask, labels=[0, 1], average='binary')
        recall_value = recall_score(true_mask, pred_mask, labels=[0, 1], average='binary')
        precision_value = precision_score(true_mask, pred_mask, labels=[0, 1], average='binary')
        intersection = np.logical_and(pred_mask, true_mask).sum()
        union = np.logical_or(pred_mask, true_mask).sum()
        iou_value = intersection / union if union > 0 else 0
        dice_value = (2. * intersection) / (np.sum(pred_mask) + np.sum(true_mask))
        pixel_accuracy = np.mean(pred_mask == true_mask)
        kappa_value = cohen_kappa_score(pred_mask, true_mask)
        mcc_value = matthews_corrcoef(pred_mask, true_mask)

        score.append([base_name, acc_value, f1_value, jac_value, recall_value, precision_value, iou_value, dice_value, pixel_accuracy, kappa_value, mcc_value])

    # Calculate average scores
    if score:
        mean_score = np.mean([s[1:] for s in score], axis=0)

        print(f"F1: {mean_score[1]:0.2f}")
        print(f"Jaccard: {mean_score[2]:0.5f}")
        print(f"Recall: {mean_score[3]:0.5f}")
        print(f"Precision: {mean_score[4]:0.5f}")

        print(f"Dice: {mean_score[6]:0.5f}")
        print(f"Pixel Accuracy: {mean_score[7]:0.5f}")
        print(f"Cohen's Kappa: {mean_score[8]:0.5f}")
        print(f"Matthews Correlation Coefficient: {mean_score[9]:0.5f}")
    else:
        print("No matching masks found")

# Test Drishti

In [118]:
cup_save_path = 'predictions/drishti/cup'

disc_save_path = 'predictions/drishti/disc'

cup_model_path = 'files/DRISHTI/z-net1-ASPP/acrima-cup.h5'

disc_model_path = 'files/DRISHTI/z-net1-ASPP/acrima-disc.h5'

segmentation_task(model_path = cup_model_path, dataset_path='datasets/drishti/test/images', save_path=cup_save_path)

segmentation_task(model_path = disc_model_path, dataset_path='datasets/drishti/test/images', save_path=disc_save_path)

Model loaded successfully
Test Images: 29


  0%|                                                                                           | 0/29 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  3%|██▊                                                                                | 1/29 [00:01<00:40,  1.46s/it]

1/1 [==============================] - 0s 27ms/step


...

1/1 [==============================] - 0s 30ms/step


 97%|███████████████████████████████████████████████████████████████████████████████▏  | 28/29 [00:03<00:00, 13.35it/s]

1/1 [==============================] - 0s 28ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:03<00:00,  7.99it/s]

segmented images saved successfully


In [119]:
model_perfromance_check(
    pred_mask_dir='predictions/drishti/disc', 
    true_mask_dir='datasets/drishti/test/masks/optic_disc_masks',
    cup_or_disc='disc')

The performance for disc's model
predicted masks 29 - true masks 29


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 21.01it/s]

F1: 0.97
Jaccard: 0.94755
Recall: 0.96919
Precision: 0.97744
Dice: 0.97292
Pixel Accuracy: 0.97379
Cohen's Kappa: 0.94654
Matthews Correlation Coefficient: 0.94724


In [120]:
model_perfromance_check(
    pred_mask_dir='predictions/drishti/cup', 
    true_mask_dir='datasets/drishti/test/masks/optic_cup_masks',
    cup_or_disc='cup')

The performance for cup's model
predicted masks 29 - true masks 29


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 16.45it/s]

F1: 0.94
Jaccard: 0.89081
Recall: 0.93683
Precision: 0.95046
Dice: 0.94174
Pixel Accuracy: 0.97094
Cohen's Kappa: 0.92160
Matthews Correlation Coefficient: 0.92329


# Test Acrima 

In [121]:
cup_save_path = 'predictions/acrima/cup'

disc_save_path = 'predictions/acrima/disc'

cup_model_path = 'files/acrima/z-net1-ASPP/acrima-cup.h5'

disc_model_path = 'files/acrima/z-net1-ASPP/acrima-disc.h5'

test_dataset = 'datasets/ACRIMA/test/images'

# Disc segmentation
segmentation_task(model_path = disc_model_path, dataset_path=test_dataset, save_path=disc_save_path)

# Cup segmentatioin
segmentation_task(model_path = cup_model_path, dataset_path=test_dataset, save_path=cup_save_path)

Model loaded successfully
Test Images: 41


  0%|                                                                                           | 0/41 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  2%|██                                                                                 | 1/41 [00:01<00:54,  1.36s/it]

1/1 [==============================] - 0s 30ms/step


...

1/1 [==============================] - 0s 26ms/step


 98%|████████████████████████████████████████████████████████████████████████████████  | 40/41 [00:04<00:00, 14.43it/s]

1/1 [==============================] - 0s 26ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [00:04<00:00,  8.80it/s]

segmented images saved successfully


In [122]:
model_perfromance_check(pred_mask_dir='predictions/acrima/disc', 
                        true_mask_dir='datasets/ACRIMA/test/masks/optic_disc_masks', 
                        cup_or_disc='disc')

The performance for disc's model
predicted masks 41 - true masks 41


100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [00:02<00:00, 15.56it/s]

F1: 0.95
Jaccard: 0.91378
Recall: 0.95393
Precision: 0.95761
Dice: 0.95450
Pixel Accuracy: 0.97917
Cohen's Kappa: 0.94074
Matthews Correlation Coefficient: 0.94178


In [123]:
model_perfromance_check(pred_mask_dir='predictions/acrima/cup', 
                        true_mask_dir='datasets/ACRIMA/test/masks/optic_cup_masks', 
                        cup_or_disc='cup')

The performance for cup's model
predicted masks 41 - true masks 41


100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [00:02<00:00, 15.37it/s]

F1: 0.90
Jaccard: 0.82127
Recall: 0.90038
Precision: 0.91134
Dice: 0.90040
Pixel Accuracy: 0.98140
Cohen's Kappa: 0.89008
Matthews Correlation Coefficient: 0.89335


# Test Rim-One

In [6]:
dataset_path = 'RIM-ONE'

In [115]:
cup_save_path = 'predictions/RIM-ONE/cup'

disc_save_path = 'predictions/RIM-ONE/disc'

cup_model_path = 'files/rim/z-net1-ASPP/acrima-cup.h5'

disc_model_path = 'files/rim/z-net1-ASPP/acrima-disc.h5'

test_dataset = 'datasets/RIM-ONE/test/images'

# Disc segmentation
segmentation_task(model_path = disc_model_path, dataset_path=test_dataset, save_path=disc_save_path)

# Cup segmentatioin
segmentation_task(model_path = cup_model_path, dataset_path=test_dataset, save_path=cup_save_path)

Model loaded successfully
Test Images: 63


  0%|                                                                                           | 0/63 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  2%|█▎                                                                                 | 1/63 [00:01<01:21,  1.32s/it]

1/1 [==============================] - 0s 27ms/step


...

1/1 [==============================] - 0s 28ms/step


 98%|████████████████████████████████████████████████████████████████████████████████▋ | 62/63 [00:06<00:00, 12.59it/s]

1/1 [==============================] - 0s 26ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 63/63 [00:06<00:00, 10.16it/s]

segmented images saved successfully


In [28]:
model_perfromance_check(pred_mask_dir='predictions/RIM-ONE/disc', 
                        true_mask_dir='datasets/RIM-ONE/test/masks/optic_disc_masks', 
                        cup_or_disc='disc')

The performance for disc's model
predicted masks 61 - true masks 61


100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [00:03<00:00, 16.99it/s]

F1: 0.97
Jaccard: 0.93687
Recall: 0.95852
Precision: 0.97720
Dice: 0.96716
Pixel Accuracy: 0.97410
Cohen's Kappa: 0.94563
Matthews Correlation Coefficient: 0.94653


In [29]:
model_perfromance_check(pred_mask_dir='predictions/RIM-ONE/cup', 
                        true_mask_dir='datasets/RIM-ONE/test/masks/optic_cup_masks', 
                        cup_or_disc='cup')

The performance for cup's model
predicted masks 61 - true masks 61


100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [00:03<00:00, 17.34it/s]

F1: 0.90
Jaccard: 0.82247
Recall: 0.92093
Precision: 0.89251
Dice: 0.90076
Pixel Accuracy: 0.98066
Cohen's Kappa: 0.88979
Matthews Correlation Coefficient: 0.89330


# Classification using ResNet-50 model 
In here we will just use the pretrained model to classifiy the images to (with optic cup, and without optic cup) the one without the optic cup will considered as normal eyes

## Constant

In [20]:
cdr_rim_normal = 'cdr/rim/normal'
cdr_rim_g = 'cdr/rim/glaucoma'

cdr_acrima_normal = 'cdr/acrima/normal'
cdr_acrima_g = 'cdr/acrima/glaucoma'

cdr_drishti_normal = 'cdr/drishti/normal'
cdr_drishti_g = 'cdr/drishti/glaucoma'

### The function to do the calssification task

In [42]:
def classification_task(images_path, output_path):
    """This function will load the classification model (Res-Net) to classifiy the images as 
    Contain optic cup and not glaucoma (there is no optic cup). The output will be saved as
    not glaucoma folder and optic-cup folder.
    Images_path : is the path for the images that we want to calssifiy,
    Output_path : is the path that we want to save our output to.
    """
    
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Flow test images in batches
    test_generator = test_datagen.flow_from_directory(
        images_path,
        target_size=(256, 256),  # Same target size as training/validation
        batch_size=32,  
        class_mode='binary', 
        shuffle=False 
    )

    # here we load the pretrained model 

    # Path to the saved model
    saved_model_path = 'classification_model/best_model.h5'  

    # Load the saved model
    model = load_model(saved_model_path)

    predictions = model.predict(test_generator, verbose=1)

    # Filenames in the same order as predictions
    filenames = test_generator.filenames
    predicted_classes = predictions > 0.5

    output_base_path = output_path

    # Directories for classified images
    optic_dir = os.path.join(output_base_path, 'optic-cup')
    no_optic_dir = os.path.join(output_base_path, 'no-glaucoma')

    # Create the directories if they don't exist
    os.makedirs(optic_dir, exist_ok=True)
    os.makedirs(no_optic_dir, exist_ok=True)

    for filename, prediction in zip(filenames, predictions):
        # Since the prediction is a probability, determine the class based on threshold
        predicted_class = "Optic" if prediction > 0.5 else "No-Optic"
        probability = prediction[0] * 100  # Convert to percentage
        print(f"{filename}: {predicted_class} - {probability:.2f}%")

        # Source file path
        src_path = os.path.join(images_path, filename)  # Ensure this is the path to your original test images

        # Determine the destination directory based on the predicted class
        if predicted_class == "Optic":
            dest_path = os.path.join(optic_dir, os.path.basename(filename))
        else:
            dest_path = os.path.join(no_optic_dir, os.path.basename(filename))

        # Copy the file to the new directory
        shutil.copy(src_path, dest_path)


### The classification process for the RIM-ONE Dataset 
1- for the normal images 

2- for the glaucoma images

In [222]:
classification_task(images_path = cdr_rim_normal, output_path = 'classified_images/rim/cdr/normal')

Found 118 images belonging to 1 classes.
4/4 [==============================] - 1s 190ms/step
normal\r1_Im001.png: Optic - 52.68%
normal\r1_Im003.png: Optic - 79.36%
normal\r1_Im004.png: Optic - 82.46%
normal\r1_Im006.png: No-Optic - 19.60%
normal\r1_Im008.png: No-Optic - 33.89%
normal\r1_Im011.png: No-Optic - 42.69%
normal\r1_Im012.png: No-Optic - 40.12%
normal\r1_Im014.png: Optic - 84.66%
normal\r1_Im015.png: Optic - 82.98%
normal\r1_Im019.png: No-Optic - 49.28%
normal\r1_Im020.png: No-Optic - 40.91%
normal\r1_Im022.png: Optic - 66.58%
normal\r1_Im023.png: Optic - 68.25%
normal\r1_Im024.png: Optic - 74.12%
normal\r1_Im025.png: Optic - 72.87%
normal\r1_Im026.png: No-Optic - 35.44%
normal\r1_Im027.png: No-Optic - 42.33%
normal\r1_Im028.png: Optic - 61.10%
normal\r1_Im029.png: No-Optic - 35.59%
normal\r1_Im031.png: No-Optic - 23.83%
normal\r1_Im032.png: No-Optic - 24.71%
normal\r1_Im033.png: No-Optic - 26.95%
normal\r1_Im036.png: Optic - 92.92%
normal\r1_Im037.png: Optic - 95.75%
normal

In [223]:
classification_task(images_path = cdr_rim_g, output_path = 'classified_images/rim/cdr/glaucoma')

Found 16 images belonging to 1 classes.
1/1 [==============================] - 1s 1s/step
glaucoma\r1_Im069.png: Optic - 93.52%
glaucoma\r1_Im103.png: No-Optic - 36.80%
glaucoma\r2_Im263.png: Optic - 66.33%
glaucoma\r2_Im264.png: Optic - 69.56%
glaucoma\r2_Im266.png: Optic - 80.70%
glaucoma\r2_Im269.png: Optic - 95.85%
glaucoma\r2_Im270.png: Optic - 70.33%
glaucoma\r2_Im289.png: Optic - 75.43%
glaucoma\r2_Im290.png: Optic - 60.01%
glaucoma\r2_Im291.png: Optic - 90.48%
glaucoma\r2_Im292.png: Optic - 63.08%
glaucoma\r2_Im305.png: Optic - 81.75%
glaucoma\r2_Im330.png: Optic - 63.85%
glaucoma\r2_Im331.png: No-Optic - 49.81%
glaucoma\r2_Im332.png: Optic - 55.97%
glaucoma\r2_Im334.png: Optic - 60.61%


### The classification process for the ACRIMA Dataset
1- for the normal images 

2- for the glaucoma images

In [188]:
classification_task(images_path = cdr_acrima_normal, output_path = 'classified_images/acrima/cdr/normal')

Found 57 images belonging to 1 classes.
2/2 [==============================] - 1s 378ms/step
normal\Im241_ACRIMA.jpg: No-Optic - 31.13%
normal\Im242_ACRIMA.jpg: No-Optic - 7.51%
normal\Im243_ACRIMA.jpg: No-Optic - 33.70%
normal\Im244_ACRIMA.jpg: No-Optic - 45.75%
normal\Im245_ACRIMA.jpg: Optic - 57.18%
normal\Im246_ACRIMA.jpg: Optic - 68.79%
normal\Im248_ACRIMA.jpg: No-Optic - 17.92%
normal\Im249_ACRIMA.jpg: No-Optic - 11.21%
normal\Im250_ACRIMA.jpg: No-Optic - 48.84%
normal\Im251_ACRIMA.jpg: No-Optic - 7.85%
normal\Im260_ACRIMA.jpg: No-Optic - 18.11%
normal\Im261_ACRIMA.jpg: Optic - 96.01%
normal\Im262_ACRIMA.jpg: Optic - 84.20%
normal\Im263_ACRIMA.jpg: No-Optic - 31.06%
normal\Im264_ACRIMA.jpg: No-Optic - 45.32%
normal\Im265_ACRIMA.jpg: Optic - 70.91%
normal\Im266_ACRIMA.jpg: No-Optic - 41.36%
normal\Im267_ACRIMA.jpg: No-Optic - 21.51%
normal\Im268_ACRIMA.jpg: No-Optic - 33.59%
normal\Im269_ACRIMA.jpg: Optic - 75.57%
normal\Im270_ACRIMA.jpg: Optic - 60.49%
normal\Im271_ACRIMA.jpg: No

In [189]:
classification_task(images_path = cdr_acrima_g, output_path = 'classified_images/acrima/cdr/glaucoma')

Found 26 images belonging to 1 classes.
1/1 [==============================] - 1s 1s/step
glaucoma\Im581_g_ACRIMA.jpg: Optic - 80.77%
glaucoma\Im590_g_ACRIMA.jpg: Optic - 92.88%
glaucoma\Im610_g_ACRIMA.jpg: Optic - 97.97%
glaucoma\Im621_g_ACRIMA.jpg: Optic - 75.60%
glaucoma\Im629_g_ACRIMA.jpg: Optic - 56.79%
glaucoma\Im632_g_ACRIMA.jpg: Optic - 99.56%
glaucoma\Im633_g_ACRIMA.jpg: Optic - 99.17%
glaucoma\Im642_g_ACRIMA.jpg: Optic - 94.80%
glaucoma\Im649_g_ACRIMA.jpg: Optic - 88.63%
glaucoma\Im651_g_ACRIMA.jpg: Optic - 92.16%
glaucoma\Im656_g_ACRIMA.jpg: Optic - 94.96%
glaucoma\Im657_g_ACRIMA.jpg: Optic - 89.57%
glaucoma\Im663_g_ACRIMA.jpg: Optic - 71.45%
glaucoma\Im666_g_ACRIMA.jpg: Optic - 90.05%
glaucoma\Im667_g_ACRIMA.jpg: Optic - 98.17%
glaucoma\Im670_g_ACRIMA.jpg: Optic - 94.70%
glaucoma\Im671_g_ACRIMA.jpg: Optic - 99.56%
glaucoma\Im675_g_ACRIMA.jpg: Optic - 98.31%
glaucoma\Im679_g_ACRIMA.jpg: Optic - 95.21%
glaucoma\Im680_g_ACRIMA.jpg: Optic - 98.58%
glaucoma\Im683_g_ACRIMA.jpg: N

### The classification process for the DRISHTI-GS dataset 
1- for the normal images 

2- for the glaucoma images

In [152]:
classification_task(images_path = cdr_drishti_normal, output_path = 'classified_images/drishti/cdr/normal')

Found 21 images belonging to 1 classes.
1/1 [==============================] - 1s 815ms/step
normal\drishtiGS_009.png: No-Optic - 30.93%
normal\drishtiGS_013.png: No-Optic - 28.84%
normal\drishtiGS_017.png: No-Optic - 39.94%
normal\drishtiGS_018.png: No-Optic - 22.78%
normal\drishtiGS_037.png: No-Optic - 32.84%
normal\drishtiGS_042.png: No-Optic - 40.43%
normal\drishtiGS_046.png: Optic - 54.39%
normal\drishtiGS_047.png: No-Optic - 26.80%
normal\drishtiGS_057.png: No-Optic - 28.10%
normal\drishtiGS_072.png: No-Optic - 36.19%
normal\drishtiGS_089.png: No-Optic - 43.19%
normal\drishtiGS_090.png: No-Optic - 45.42%
normal\drishtiGS_091.png: Optic - 51.73%
normal\drishtiGS_093.png: No-Optic - 41.92%
normal\drishtiGS_094.png: No-Optic - 35.14%
normal\drishtiGS_095.png: Optic - 56.73%
normal\drishtiGS_096.png: No-Optic - 25.22%
normal\drishtiGS_097.png: Optic - 82.47%
normal\drishtiGS_099.png: Optic - 60.14%
normal\drishtiGS_100.png: No-Optic - 25.91%
normal\drishtiGS_101.png: No-Optic - 37.88

In [153]:
classification_task(images_path = cdr_drishti_g, output_path = 'classified_images/drishti/cdr/glaucoma')

Found 15 images belonging to 1 classes.
1/1 [==============================] - 1s 973ms/step
glaucoma\drishtiGS_004.png: Optic - 59.68%
glaucoma\drishtiGS_010.png: Optic - 96.60%
glaucoma\drishtiGS_012.png: Optic - 57.10%
glaucoma\drishtiGS_024.png: Optic - 54.89%
glaucoma\drishtiGS_031.png: No-Optic - 25.50%
glaucoma\drishtiGS_032.png: Optic - 97.24%
glaucoma\drishtiGS_058.png: Optic - 72.27%
glaucoma\drishtiGS_060.png: Optic - 77.53%
glaucoma\drishtiGS_061.png: Optic - 80.69%
glaucoma\drishtiGS_063.png: Optic - 97.86%
glaucoma\drishtiGS_066.png: Optic - 51.03%
glaucoma\drishtiGS_068.png: Optic - 89.46%
glaucoma\drishtiGS_069.png: Optic - 83.16%
glaucoma\drishtiGS_080.png: Optic - 56.02%
glaucoma\drishtiGS_084.png: Optic - 59.10%


# segment the images that contain optic-cup (output of the classification)

## Drishti 

### Normal

In [154]:
test_dataset = 'classified_images/drishti/cdr/normal/optic-cup'

cup_save_path = 'classified_images/drishti/segmentation/normal/cup'
disc_save_path = 'classified_images/drishti/segmentation/normal/disc'

disc_model_path = 'files/DRISHTI/z-net1-ASPP/acrima-disc.h5'
cup_model_path = 'files/DRISHTI/z-net1-ASPP/acrima-cup.h5'


# Disc segmentation
segmentation_task(model_path = disc_model_path, dataset_path=test_dataset, save_path=disc_save_path)

# Cup segmentation 
segmentation_task(model_path = cup_model_path, dataset_path=test_dataset, save_path=cup_save_path)



Model loaded successfully
Test Images: 5


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


 20%|████████████████▊                                                                   | 1/5 [00:01<00:05,  1.33s/it]

1/1 [==============================] - 0s 27ms/step


...

1/1 [==============================] - 0s 26ms/step


 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:01<00:00,  2.60it/s]

1/1 [==============================] - 0s 30ms/step


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.01it/s]

segmented images saved successfully


### Glaucoma

In [155]:
test_dataset = 'classified_images/drishti/cdr/glaucoma/optic-cup'

cup_save_path = 'classified_images/drishti/segmentation/glaucoma/cup'
disc_save_path = 'classified_images/drishti/segmentation/glaucoma/disc'


# Disc segmentation
segmentation_task(model_path = disc_model_path, dataset_path=test_dataset, save_path=disc_save_path)

# Cup segmentation 
segmentation_task(model_path = cup_model_path, dataset_path=test_dataset, save_path=cup_save_path)



Model loaded successfully
Test Images: 14


  0%|                                                                                           | 0/14 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  7%|█████▉                                                                             | 1/14 [00:01<00:16,  1.29s/it]

1/1 [==============================] - 0s 26ms/step


...

1/1 [==============================] - 0s 28ms/step


 93%|████████████████████████████████████████████████████████████████████████████▏     | 13/14 [00:02<00:00,  9.01it/s]

1/1 [==============================] - 0s 26ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:02<00:00,  5.26it/s]

segmented images saved successfully


# CDR Code  

In [144]:
def CDR(optic_cup_dir, optic_disc_dir):
    def count_white_pixels(image_path):
        # Load the image in grayscale mode
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        # Check if the image was loaded correctly
        if image is not None:
            # Count the white pixels
            return np.sum(image == 255)
        else:
            print(f"Error loading the image {os.path.basename(image_path)}")
            return None

    # List all the optic cup and optic disc image files (ensure they are in the same order)
    optic_cup_files = sorted(glob(os.path.join(optic_cup_directory, '*')))
    optic_disc_files = sorted(glob(os.path.join(optic_disc_directory, '*')))

    # Ensure that every optic cup has a corresponding optic disc
    assert len(optic_cup_files) == len(optic_disc_files), "The number of optic cup and disc images must match"

    cdr_values = {}
    results = {}

    # Calculate CDR and classify as glaucoma or normal
    for cup_path, disc_path in zip(optic_cup_files, optic_disc_files):
        cup_pixels = count_white_pixels(cup_path)
        disc_pixels = count_white_pixels(disc_path)
        if cup_pixels is not None and disc_pixels is not None:
            cdr = cup_pixels / disc_pixels if disc_pixels > 0 else 0
            image_name = os.path.basename(cup_path)
            cdr_values[image_name] = cdr
            results[image_name] = "Glaucoma" if cdr >= 0.4 else "Normal"
            print(f"{image_name} - CDR: {cdr:.2f} - Result: {results[image_name]}")

# CDR Accuracy matric

In [162]:
def cdr_accuracy(total_images, correct_predictions, dataset_name):
    accuracy = (correct_predictions/total_images) * 100
    print(f'The CDR Accuracy for {dataset_name} dataset is {accuracy:0.4f}')

### CDR for Normal (drishti)

In [156]:
optic_cup_directory = 'classified_images/drishti/segmentation/normal/cup'
optic_disc_directory  = 'classified_images/drishti/segmentation/normal/disc'

In [157]:
CDR(optic_cup_directory, optic_disc_directory)

drishtiGS_046.png - CDR: 0.34 - Result: Normal
drishtiGS_091.png - CDR: 0.31 - Result: Normal
drishtiGS_095.png - CDR: 0.37 - Result: Normal
drishtiGS_097.png - CDR: 0.33 - Result: Normal
drishtiGS_099.png - CDR: 0.42 - Result: Glaucoma


### CDR for Glaucoma (drishti)

In [158]:
optic_cup_directory = 'classified_images/drishti/segmentation/glaucoma/cup'
optic_disc_directory  = 'classified_images/drishti/segmentation/glaucoma/disc'

In [159]:
CDR(optic_cup_directory, optic_disc_directory)

drishtiGS_004.png - CDR: 0.49 - Result: Glaucoma
drishtiGS_010.png - CDR: 0.80 - Result: Glaucoma
drishtiGS_012.png - CDR: 0.69 - Result: Glaucoma
drishtiGS_024.png - CDR: 0.51 - Result: Glaucoma
drishtiGS_032.png - CDR: 0.58 - Result: Glaucoma
drishtiGS_058.png - CDR: 0.61 - Result: Glaucoma
drishtiGS_060.png - CDR: 0.53 - Result: Glaucoma
drishtiGS_061.png - CDR: 0.65 - Result: Glaucoma
drishtiGS_063.png - CDR: 0.65 - Result: Glaucoma
drishtiGS_066.png - CDR: 0.49 - Result: Glaucoma
drishtiGS_068.png - CDR: 0.62 - Result: Glaucoma
drishtiGS_069.png - CDR: 0.66 - Result: Glaucoma
drishtiGS_080.png - CDR: 0.54 - Result: Glaucoma
drishtiGS_084.png - CDR: 0.51 - Result: Glaucoma


In [230]:
cdr_accuracy(total_images = 36, correct_predictions=34, dataset_name='drishti')

The CDR Accuracy for drishti dataset is 94.4444


## Acrima

### Normal

In [190]:
test_dataset = 'classified_images/acrima/cdr/normal/optic-cup'

cup_save_path = 'classified_images/acrima/segmentation/normal/cup'
disc_save_path = 'classified_images/acrima/segmentation/normal/disc'

disc_model_path = 'files/acrima/z-net1-ASPP/acrima-disc.h5'
cup_model_path = 'files/acrima/z-net1-ASPP/acrima-cup.h5'


# Disc segmentation
segmentation_task(model_path = disc_model_path, dataset_path=test_dataset, save_path=disc_save_path)

# Cup segmentation 
segmentation_task(model_path = cup_model_path, dataset_path=test_dataset, save_path=cup_save_path)



Model loaded successfully
Test Images: 26


  0%|                                                                                           | 0/26 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  4%|███▏                                                                               | 1/26 [00:01<00:31,  1.28s/it]

1/1 [==============================] - 0s 27ms/step


...

1/1 [==============================] - 0s 30ms/step


 92%|███████████████████████████████████████████████████████████████████████████▋      | 24/26 [00:03<00:00, 12.30it/s]

1/1 [==============================] - 0s 27ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:03<00:00,  7.37it/s]

segmented images saved successfully


### Glaucoma

In [191]:
test_dataset = 'classified_images/acrima/cdr/glaucoma/optic-cup'

cup_save_path = 'classified_images/acrima/segmentation/glaucoma/cup'
disc_save_path = 'classified_images/acrima/segmentation/glaucoma/disc'


# Disc segmentation
segmentation_task(model_path = disc_model_path, dataset_path=test_dataset, save_path=disc_save_path)

# Cup segmentation 
segmentation_task(model_path = cup_model_path, dataset_path=test_dataset, save_path=cup_save_path)



Model loaded successfully
Test Images: 25


  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  4%|███▎                                                                               | 1/25 [00:01<00:30,  1.28s/it]

1/1 [==============================] - 0s 26ms/step


...

1/1 [==============================] - 0s 26ms/step


 96%|██████████████████████████████████████████████████████████████████████████████▋   | 24/25 [00:04<00:00, 10.08it/s]

1/1 [==============================] - 0s 35ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:04<00:00,  6.05it/s]

segmented images saved successfully


### CDR for Normal (Acrima)

In [192]:
optic_cup_directory = 'classified_images/acrima/segmentation/normal/cup'
optic_disc_directory  = 'classified_images/acrima/segmentation/normal/disc'

CDR(optic_cup_directory, optic_disc_directory)

Im245_ACRIMA.jpg - CDR: 0.20 - Result: Normal
Im246_ACRIMA.jpg - CDR: 0.15 - Result: Normal
Im261_ACRIMA.jpg - CDR: 0.18 - Result: Normal
Im262_ACRIMA.jpg - CDR: 0.17 - Result: Normal
Im265_ACRIMA.jpg - CDR: 0.21 - Result: Normal
Im269_ACRIMA.jpg - CDR: 0.14 - Result: Normal
Im270_ACRIMA.jpg - CDR: 0.15 - Result: Normal
Im276_ACRIMA.jpg - CDR: 0.15 - Result: Normal
Im281_ACRIMA.jpg - CDR: 0.27 - Result: Normal
Im282_ACRIMA.jpg - CDR: 0.20 - Result: Normal
Im284_ACRIMA.jpg - CDR: 0.18 - Result: Normal
Im285_ACRIMA.jpg - CDR: 0.21 - Result: Normal
Im286_ACRIMA.jpg - CDR: 0.34 - Result: Normal
Im288_ACRIMA.jpg - CDR: 0.25 - Result: Normal
Im290_ACRIMA.jpg - CDR: 0.33 - Result: Normal
Im291_ACRIMA.jpg - CDR: 0.17 - Result: Normal
Im293_ACRIMA.jpg - CDR: 0.13 - Result: Normal
Im295_ACRIMA.jpg - CDR: 0.22 - Result: Normal
Im298_ACRIMA.jpg - CDR: 0.25 - Result: Normal
Im299_ACRIMA.jpg - CDR: 0.26 - Result: Normal
Im300_ACRIMA.jpg - CDR: 0.10 - Result: Normal
Im301_ACRIMA.jpg - CDR: 0.16 - Res

### CDR for Glaucoma (Acrima)

In [193]:
optic_cup_directory = 'classified_images/acrima/segmentation/glaucoma/cup'
optic_disc_directory  = 'classified_images/acrima/segmentation/glaucoma/disc'

CDR(optic_cup_directory, optic_disc_directory)

Im581_g_ACRIMA.jpg - CDR: 0.47 - Result: Glaucoma
Im590_g_ACRIMA.jpg - CDR: 0.54 - Result: Glaucoma
Im610_g_ACRIMA.jpg - CDR: 0.46 - Result: Glaucoma
Im621_g_ACRIMA.jpg - CDR: 0.43 - Result: Glaucoma
Im629_g_ACRIMA.jpg - CDR: 0.51 - Result: Glaucoma
Im632_g_ACRIMA.jpg - CDR: 0.54 - Result: Glaucoma
Im633_g_ACRIMA.jpg - CDR: 0.66 - Result: Glaucoma
Im642_g_ACRIMA.jpg - CDR: 0.64 - Result: Glaucoma
Im649_g_ACRIMA.jpg - CDR: 0.53 - Result: Glaucoma
Im651_g_ACRIMA.jpg - CDR: 0.61 - Result: Glaucoma
Im656_g_ACRIMA.jpg - CDR: 0.39 - Result: Normal
Im657_g_ACRIMA.jpg - CDR: 0.43 - Result: Glaucoma
Im663_g_ACRIMA.jpg - CDR: 0.43 - Result: Glaucoma
Im666_g_ACRIMA.jpg - CDR: 0.57 - Result: Glaucoma
Im667_g_ACRIMA.jpg - CDR: 0.60 - Result: Glaucoma
Im670_g_ACRIMA.jpg - CDR: 0.41 - Result: Glaucoma
Im671_g_ACRIMA.jpg - CDR: 0.48 - Result: Glaucoma
Im675_g_ACRIMA.jpg - CDR: 0.57 - Result: Glaucoma
Im679_g_ACRIMA.jpg - CDR: 0.62 - Result: Glaucoma
Im680_g_ACRIMA.jpg - CDR: 0.67 - Result: Glaucoma
Im

In [194]:
cdr_accuracy(total_images = 84, correct_predictions=81, dataset_name='acrima')

The CDR Accuracy for acrima dataset is 96.4286


## Rim-One

### Normal

In [224]:
test_dataset = 'classified_images/rim/cdr/normal/optic-cup'

cup_save_path = 'classified_images/rim/segmentation/normal/cup'
disc_save_path = 'classified_images/rim/segmentation/normal/disc'

disc_model_path = 'files/rim/z-net1-ASPP/acrima-disc.h5'
cup_model_path = 'files/rim/z-net1-ASPP/acrima-cup.h5'


# Disc segmentation
segmentation_task(model_path = disc_model_path, dataset_path=test_dataset, save_path=disc_save_path)

# Cup segmentation 
segmentation_task(model_path = cup_model_path, dataset_path=test_dataset, save_path=cup_save_path)



Model loaded successfully
Test Images: 47


  0%|                                                                                           | 0/47 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  2%|█▊                                                                                 | 1/47 [00:01<01:01,  1.34s/it]

1/1 [==============================] - 0s 26ms/step


...

1/1 [==============================] - 0s 27ms/step


 96%|██████████████████████████████████████████████████████████████████████████████▌   | 45/47 [00:05<00:00, 11.29it/s]

1/1 [==============================] - 0s 26ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:05<00:00,  9.06it/s]

segmented images saved successfully


### Glaucoma

In [225]:
test_dataset = 'classified_images/rim/cdr/glaucoma/optic-cup'

cup_save_path = 'classified_images/rim/segmentation/glaucoma/cup'
disc_save_path = 'classified_images/rim/segmentation/glaucoma/disc'


# Disc segmentation
segmentation_task(model_path = disc_model_path, dataset_path=test_dataset, save_path=disc_save_path)

# Cup segmentation 
segmentation_task(model_path = cup_model_path, dataset_path=test_dataset, save_path=cup_save_path)



Model loaded successfully
Test Images: 14


  0%|                                                                                           | 0/14 [00:00<?, ?it/s]

1/1 [==============================] - 2s 2s/step


  7%|█████▉                                                                             | 1/14 [00:01<00:22,  1.77s/it]

1/1 [==============================] - 0s 27ms/step


...

1/1 [==============================] - 0s 27ms/step


 93%|████████████████████████████████████████████████████████████████████████████▏     | 13/14 [00:02<00:00,  9.09it/s]

1/1 [==============================] - 0s 25ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:02<00:00,  5.49it/s]

segmented images saved successfully


### CDR for normal (Rim-One)

In [226]:
optic_cup_directory = 'classified_images/rim/segmentation/normal/cup'
optic_disc_directory  = 'classified_images/rim/segmentation/normal/disc'

CDR(optic_cup_directory, optic_disc_directory)

r1_Im001.png - CDR: 0.20 - Result: Normal
r1_Im003.png - CDR: 0.29 - Result: Normal
r1_Im004.png - CDR: 0.36 - Result: Normal
r1_Im014.png - CDR: 0.38 - Result: Normal
r1_Im015.png - CDR: 0.35 - Result: Normal
r1_Im022.png - CDR: 0.21 - Result: Normal
r1_Im023.png - CDR: 0.19 - Result: Normal
r1_Im024.png - CDR: 0.25 - Result: Normal
r1_Im025.png - CDR: 0.22 - Result: Normal
r1_Im028.png - CDR: 0.30 - Result: Normal
r1_Im036.png - CDR: 0.44 - Result: Glaucoma
r1_Im037.png - CDR: 0.37 - Result: Normal
r1_Im040.png - CDR: 0.28 - Result: Normal
r1_Im041.png - CDR: 0.25 - Result: Normal
r1_Im050.png - CDR: 0.38 - Result: Normal
r1_Im052.png - CDR: 0.29 - Result: Normal
r1_Im054.png - CDR: 0.17 - Result: Normal
r1_Im056.png - CDR: 0.30 - Result: Normal
r1_Im059.png - CDR: 0.29 - Result: Normal
r1_Im060.png - CDR: 0.34 - Result: Normal
r1_Im061.png - CDR: 0.19 - Result: Normal
r1_Im062.png - CDR: 0.19 - Result: Normal
r1_Im063.png - CDR: 0.17 - Result: Normal
r1_Im087.png - CDR: 0.23 - Resul

### CDR for glaucoma (Rim-One)

In [227]:
optic_cup_directory = 'classified_images/rim/segmentation/glaucoma/cup'
optic_disc_directory  = 'classified_images/rim/segmentation/glaucoma/disc'

CDR(optic_cup_directory, optic_disc_directory)

r1_Im069.png - CDR: 0.61 - Result: Glaucoma
r2_Im263.png - CDR: 0.65 - Result: Glaucoma
r2_Im264.png - CDR: 0.54 - Result: Glaucoma
r2_Im266.png - CDR: 0.30 - Result: Normal
r2_Im269.png - CDR: 0.41 - Result: Glaucoma
r2_Im270.png - CDR: 0.45 - Result: Glaucoma
r2_Im289.png - CDR: 0.56 - Result: Glaucoma
r2_Im290.png - CDR: 0.32 - Result: Normal
r2_Im291.png - CDR: 0.38 - Result: Normal
r2_Im292.png - CDR: 0.37 - Result: Normal
r2_Im305.png - CDR: 0.44 - Result: Glaucoma
r2_Im330.png - CDR: 0.42 - Result: Glaucoma
r2_Im332.png - CDR: 0.24 - Result: Normal
r2_Im334.png - CDR: 0.36 - Result: Normal


In [228]:
cdr_accuracy(total_images = 134, correct_predictions=125, dataset_name='acrima')

The CDR Accuracy for acrima dataset is 93.2836
